#### Imports


In [ ]:
import pandas as pd
import gradio as gr
from autogluon.tabular import TabularPredictor
import joblib
import numpy as np
import plotly.graph_objects as go

#### Model Loading and Inference


In [ ]:
def predict_fire_area(
    estimated_fire_brightness,
    estimated_fire_radiative_power,
    precipitation,
    relative_humidity,
    soil_water_content,
    solar_radiation,
    temperature,
    wind_speed,
):
    # Load model
    predictor = TabularPredictor.load("/workspace/autogluonModels/iter6")

    # Check if any of the inputs are None
    if (
        estimated_fire_brightness == "None"
        or estimated_fire_radiative_power == "None"
        or precipitation == "None"
        or relative_humidity == "None"
        or soil_water_content == "None"
        or solar_radiation == "None"
        or temperature == "None"
        or wind_speed == "None"
    ):
        return "None"

    # Convert interface inputs to dataframe
    input_dict = pd.DataFrame.from_dict(
        {
            "Mean_estimated_fire_brightness": [estimated_fire_brightness],
            "Mean_estimated_fire_radiative_power": [estimated_fire_radiative_power],
            "Precipitation_PCA": [precipitation],
            "RelativeHumidity_PCA": [relative_humidity],
            "SoilWaterContent_PCA": [soil_water_content],
            "SolarRadiation_PCA": [solar_radiation],
            "Temperature_PCA": [temperature],
            "WindSpeed_PCA": [wind_speed],
        }
    )

    # Scale inputs
    train_scaler = joblib.load("/workspace/train_scaler.pkl")
    test_scaler = joblib.load("/workspace/test_scaler.pkl")
    input_val = train_scaler.transform(input_dict)
    input_val = pd.DataFrame(input_val, columns=input_dict.columns)
    pred = predictor.predict(input_val)
    out = np.array([pred]).reshape(-1, 1)
    out = test_scaler.inverse_transform(out)

    # Given areas
    areas = [8451.287070, 146.434093, 38.218684, abs(out[0][0])]

    # Calculating side lengths for each square
    side_lengths = [a**0.5 for a in areas]

    # Define function to get the square vertices
    def get_square_vertices(side_length):
        half_side = side_length / 2
        return [
            [-half_side, -half_side],
            [half_side, -half_side],
            [half_side, half_side],
            [-half_side, half_side],
            [-half_side, -half_side],
        ]

    # Get vertices for each square
    vertices = [get_square_vertices(s) for s in side_lengths]

    # Plotting
    fig = go.Figure()

    for vert, area in zip(vertices, areas):
        x_vals, y_vals = zip(*vert)
        fig.add_trace(
            go.Scatter(x=x_vals, y=y_vals, fill="toself", name=f"Area: {area}")
        )

    return abs(out[0][0]), fig

#### Building Gradio Interface


In [ ]:
# Interface
inputs = [
    gr.Number(value=321.258978, label="Mean Estimated Fire Brightness"),
    gr.Number(value=87.642789, label="Mean Estimated Fire Radiative Power"),
    gr.Slider(0.013398, 8.105487, label="Precipitation"),
    gr.Slider(0.014405, 5.198580, label="Relative Humidity"),
    gr.Slider(0.029377, 5.032946, label="Soil Water Content"),
    gr.Slider(0.045974, 4.603705, label="Solar Radiation"),
    gr.Slider(0.024974, 4.756035, label="Temperature"),
    gr.Slider(0.025324, 4.802098, label="Wind Speed"),
]

outputs = [gr.Text(label="Estimated Fire Area"), gr.Plot(label="Area Visualization")]

# Create interface
demo = gr.Interface(
    predict_fire_area,
    inputs,
    outputs=outputs,
    live=True,
    allow_flagging="never",
    title="Predicting Wildfires with Weather Forecast Data",
)

#### Launching Gradio Interface


In [ ]:
demo.launch()